## Analyzor Notebook for CEX-DEX Arbs and Competing Transactions in the Mempool

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import math
import seaborn as sns


In [ ]:

# Range: 17195493-17195495
target_pool = pd.read_csv('target_pool.csv')

# Find all transactions going to the cex-dex arber
cex_dex_arber = '0xa69babef1ca67a37ffaf7a485dfff3382056e78c'
arber_txs = target_pool[target_pool['toaddress'] == cex_dex_arber]

## Mempooldan bu bota giden transactionlari aldik
## Amac: ayni mempoolda olup kazanamayan baska adresleri bul
## fakat bu adresler de 


# Group the transactions by hash and for each unique hash, find the set of observed status and corresponding detecttime
arber_txs_grouped = arber_txs.groupby('hash').agg({'status': lambda x: set(x), 'detecttime': lambda x: set(x)})


In [ ]:
def plot_tx_data_in_current_block(block_number):
    cur_block_txs = target_pool[target_pool['curblocknumber'] == block_number]

    # Group the data by region and create new dataframes for each group
    grouped_data = cur_block_txs.groupby('region')

    # Create a dictionary to store the dataframes
    txs_regions_dict = {}

    # Loop through the groups and create new dataframes for each group
    for region, data in grouped_data:
        df_name = f'{region}_txs'
        txs_regions_dict[df_name] = data.copy()
        print(f'Created dataframe: {df_name}')

    # accessing the regions:
    # south_east = txs_regions_dict['ap-southeast-1_txs']
    # eu_central = txs_regions_dict['eu-central-1_txs']
    # us_east = txs_regions_dict['us-east-1_txs']

    region = txs_regions_dict['us-east-1_txs']

     # aggregated = us_east.groupby('hash', as_index=False).agg({'status': lambda x: set(x), 'detecttime': lambda x: set(x)})
    status_to_color_plot = {
    'pending': 'olive', #yellow
    'confirmed': 'green', #green
    'rejected': 'red', #red
    'cancelled': 'black', #black
    'failed': 'gray', #gray
    'evicted': 'blue', #navyblue
    'speedup': 'purple' #purple
    }
    region['color'] = region['status'].apply(lambda x: status_to_color_plot.get(x, "pink"))
    region['detecttimemilis'] = pd.to_datetime(region['detecttime']).astype(int) / 10**6
    region['gasprice_gwei'] = region['gasprice'] * 1e-9
    region['gaspaid_gwei'] = region['gas'] * 1e-9
    region['normalized_milis'] = region['detecttimemilis'] - region['detecttimemilis'].min()

    scatterPlot, ax = plt.subplots()
    handles = []
    for status, color in status_to_color_plot.items():
        scale = 100.0
        handle = ax.scatter(x=region['normalized_milis'], y=region['gasprice_gwei'], c=region['color'], s=scale, label=status,
               alpha=0.3, edgecolors='none')
        handles.append(mpatches.Patch(color=color, label=status))

    ax.legend(handles=handles)
    ax.grid(True)

    # set the x-axis label
    ax.set_xlabel('Detect Time (miliseconds)')

    # set the y-axis label
    ax.set_ylabel('Gas Price (gwei)')

    # Add title
    numConfirmed = len(region[region['status'] == 'confirmed'])
    ax.set_title("Current block: " + str(region['curblocknumber'].iloc[0]) + "\nConfirmed Txs: " + str(numConfirmed))

In [ ]:
target_pool.columns

In [ ]:
arber_txs_grouped

In [ ]:
arber_txs_grouped.shape[0]

In [ ]:
result = target_pool[target_pool["input"].str.contains('6f70d6b5f4404a0618cad9c1987d35f5ea79e529')]
## bu string parcasi, ikinci adres uzerinde cagirilan fonksiyonun byte codeu
## herhangi bir public transaction olsaydi status'ta sadece confirmed gormezdik, pending de gorurduk. hic pend etmeden inclue olmus, nasil oldu gokten mi indi? evet gokten indi cunku private order flow

print(result)

In [ ]:
# bytes'a bakarak adam hangi poolla interact etmis olabilir?
# bu kazananin byte code'unu butun transactionlarin byte kodlariyla karsilastir. input'a bakabilirsin
bytes = 0x588116d8000000000000000000000000000000000000000000000000000003e122d5eb1c0000000000000000000000000000000000000000000000001b7e48d691ce910000000000000000000000000000000000000057c8b3be32320daad0b2421fb41a0000000000000000000000000000000000000000000000000de0b6b3a764000000000000000000000000000000000000000000000000000000000000645528e2ff000000000000000000000000000000000000000000000000000000000279d7

In [ ]:
num_nan = target_pool['gasprice'].isna().sum()
print(f"Number of NaN values in 'gasprice' column: {num_nan}")
print("Number of rows in target pool is " + str(target_pool.shape[0]))

In [ ]:
plot_tx_data_in_current_block(17195495)

In [ ]:
arber_txs_grouped

In [ ]:
target_pool.columns

In [ ]:
target_pool[target_pool['status'] == 'pending']

someDf = target_pool.groupby('hash').agg({'status': lambda x: set(x), 'detecttime': lambda x: set(x), 'curblocknumber': lambda x: set(x)}).reset_index()
txDf = someDf[someDf['hash'].str.contains('0x0050cd6b561ec7af20b6e3daa561dbd71b4a2360')]
someDf

In [ ]:
# Iterate over columns in pandas DataFrame using enumerate()
for (index, colname) in enumerate(txDf):
    print(index, txDf[colname].values)
    

In [ ]:
txDf.to_csv('filename.csv')

In [ ]:
target_pool[target_pool['hash'] == '0x030e12b671f93138c92d0ed70e0a51b8720db091e0ce17f29b0efbcdb626fac2'].to_csv('tx2.csv')

In [ ]:
# Read the CSV file with tabs as separator
new_pool = pd.read_csv('new_pool.csv', sep='\t')

# Remove leading/trailing spaces from column names
new_pool.columns = new_pool.columns.str.strip()

In [ ]:
new_pool.columns

In [ ]:
print(new_pool['curblocknumber'])
print(len(new_pool))

In [ ]:
ten_block_df = new_pool[(new_pool['curblocknumber'] >= 17195490) & (new_pool['curblocknumber'] <= 17195500)]
block_mask = ten_block_df['curblocknumber'] == 17195490
block_range_mask = (ten_block_df['curblocknumber'] >= 17195494) & (ten_block_df['curblocknumber'] <= 17195496)
confirmed_status_mask = ten_block_df['status'] == 'confirmed'
evicted_status_mask = ten_block_df['status'] == 'evicted'
pending_status_mask = ten_block_df['status'] == 'pending'
combined_status_mask = confirmed_status_mask | pending_status_mask
confirmed_df = ten_block_df.loc[block_range_mask & combined_status_mask, ['curblocknumber', 'status', 'detecttime']]

# get the minimum time
confirmed_df['detecttime_milis'] = pd.to_datetime(confirmed_df['detecttime']).astype(int) / 10**6
confirmed_df['normalized_milis'] = confirmed_df['detecttime_milis'] - confirmed_df['detecttime_milis'].min()
# confirmed_df['region'] = confirmed_df[confirmed_df['region'] == 'us-east-1']

num_unique_detecttime = confirmed_df['detecttime'].nunique()
print("unique timestamp values: " + str(num_unique_detecttime))


In [ ]:
def plot_blocks_statuses(df):
    # create a dictionary to map status to color
    color_map = {'pending': 'orange', 'confirmed': 'green', 'rejected': 'red', 'cancel': 'purple', 'failed': 'yellow', 'evicted': 'blue', 'speedup': 'gray'}
    # add new columns detecttimemillis and gasprice_gwei
    df['detecttimemillis'] = pd.to_datetime(df['detecttime']).astype(int) // 10**6
    df['gasprice_gwei'] = df['gasprice'] * 1e-9
    df = df[(df['gasprice_gwei'] <= 400) & (df['status'] != 'evicted')]
    # plot each point with its associated color
    for status, color in color_map.items():
        status_df = df[df['status'] == status]
        plt.scatter(status_df['detecttimemillis'], status_df['gasprice_gwei'], c=color, label=status)
    # loop through each block and plot a vertical line for the earliest confirmed point
    for block in df['curblocknumber'].unique():
        block_df = df[df['curblocknumber'] == block]
        earliest_confirmed = block_df[block_df['status'] == 'confirmed']['detecttimemillis'].min()
        plt.axvline(x=earliest_confirmed, color='black', linestyle='--')
        plt.text(earliest_confirmed, df['gasprice_gwei'].max(), f'{block}', ha='center')
    # set the x-axis label and title
    plt.xlabel('Timestamp (Miliseconds)')
    plt.title('Gas Price (Gwei) vs Timestamp')
    # set the x-axis range to start from the lowest timestamp in the dataframe
    plt.xlim(left=df['detecttimemillis'].min())
    # add the legend at the end by color
    handles = [plt.plot([], [], marker='o', ls="", color=color_map[label], label=label)[0] for label in color_map.keys()]
    plt.legend(handles=handles, loc='upper right')
    # set the plot dimensions to make the x-axis longer than
    plt.gcf().set_size_inches(15, 5)

    # show the plot
    plt.show()


In [ ]:
# block_df = new_pool[(new_pool['curblocknumber'] >= 17195490) & (new_pool['curblocknumber'] <= 17195496)].copy()
block_df = new_pool[(new_pool['curblocknumber'] >= 17195490) & (new_pool['curblocknumber'] <= 17195500) & (new_pool['region'] == 'us-east-1')].copy()
plot_blocks_statuses(block_df)

In [ ]:
def plot_blocks_mempool_type(df):
    # create a dictionary to map status to color
    color_map = {'private': 'red', 'public': 'blue'}
    # add new columns detecttimemillis and gasprice_gwei
    df['detecttimemillis'] = pd.to_datetime(df['detecttime']).astype(int) // 10**6
    df['gasprice_gwei'] = df['gasprice'] * 1e-9
    df = df[(df['gasprice_gwei'] <= 400) & (df['status'] != 'evicted')]
    # plot each point with its associated color
    for flow, color in color_map.items():
        filtered_df = df[(df['order_flow'] == flow) & (df['status'] == 'confirmed')]
        plt.scatter(filtered_df['detecttimemillis'], filtered_df['gasprice_gwei'], c=color, label=flow)
    # loop through each block and plot a vertical line for the earliest confirmed point
    for block in df['curblocknumber'].unique():
        block_df = df[df['curblocknumber'] == block]
        print(f'Block: {block}')
        earliest_confirmed = block_df[block_df['status'] == 'confirmed']['detecttimemillis'].min()
        print(f'Earliest Confifmed: {earliest_confirmed}\n')
        plt.axvline(x=earliest_confirmed, color='black', linestyle='--')
        plt.text(earliest_confirmed, df['gasprice_gwei'].max(), f'{block}', ha='center')
    # set the x-axis label and title
    plt.xlabel('Timestamp (Miliseconds)')
    plt.title('Gas Price (Gwei) vs Timestamp for Public vs. Private Order Flows')
    # set the x-axis range to start from the lowest timestamp in the dataframe
    plt.xlim(left=df['detecttimemillis'].min())
    # add the legend at the end by color
    handles = [plt.plot([], [], marker='o', ls="", color=color_map[label], label=label)[0] for label in color_map.keys()]
    plt.legend(handles=handles, loc='upper right')
    # set the plot dimensions to make the x-axis longer than
    plt.gcf().set_size_inches(15, 5)

    # show the plot
    plt.show()


In [ ]:
p_block_df = block_df.copy()
p_block_df['order_flow'] = 'private'
filtered = p_block_df[p_block_df['status'] == 'pending']
p_block_df.loc[p_block_df['hash'].isin(filtered['hash']), 'order_flow'] = 'public'
plot_blocks_mempool_type(p_block_df)

In [ ]:
grouped_status_df = block_df.groupby('hash').agg({'status': lambda x: set(x), 'detecttime': lambda x: set(x)}).reset_index()
grouped_status_df

color_map = {'private': 'red', 'public': 'blue'}

def get_order_flow(status_set):
    if 'pending' in status_set:
        return 'public'
    else:
        return 'private'

grouped_status_df['order_flow'] = grouped_status_df['status'].apply(get_order_flow)
grouped_status_df['color'] = grouped_status_df['order_flow'].map(color_map)
grouped_status_df

exploded_df = grouped_status_df.explode('detecttime').explode('status')
exploded_df

In [ ]:
def plot_confirmed_txs(df):
    # create a dictionary to map status to color
    color_map = {'private': 'red', 'public': 'blue'}
    # add new columns detecttimemillis and gasprice_gwei
    df['detecttimemillis'] = pd.to_datetime(df['detecttime']).astype(int) // 10**6
    df['gasprice_gwei'] = df['gasprice'] * 1e-9
    # plot each point with its associated color and size based on gasprice_gwei
    for flow, color in color_map.items():
        filtered_df = df[df['order_flow'] == flow]
        plt.scatter(filtered_df['detecttimemillis'], filtered_df['timepending'], c=color, label=flow, s=filtered_df['gasprice_gwei'])
    # loop through each block and plot a vertical line for the earliest confirmed point
    for block in df['curblocknumber'].unique():
        block_df = df[df['curblocknumber'] == block]
        earliest_confirmed = block_df[block_df['status'] == 'confirmed']['detecttimemillis'].min()
        plt.axvline(x=earliest_confirmed, color='black', linestyle='--')
        # plt.text(earliest_confirmed, df['gasprice_gwei'].max(), f'{block}', ha='center')
    # set the x-axis label and title
    plt.xlabel('Timestamp (Miliseconds)')
    plt.title('Inclusion time (Miliseconds) vs Time (Miliseconds)')
    # set the x-axis range to start from the lowest timestamp in the dataframe
    plt.xlim(left=df['detecttimemillis'].min())
    # add the legend at the end by color
    handles = [plt.plot([], [], marker='o', ls="", color=color_map[label], label=label)[0] for label in color_map.keys()]
    plt.legend(handles=handles, loc='upper right')
    # set the plot dimensions to make the x-axis longer than
    plt.gcf().set_size_inches(15, 5)

    # show the plot
    plt.show()

In [ ]:
def heatmap_gas_vs_inclusion(df):

    # Create a heatmap with inclusion time on the x-axis and total gas spent in block on the y-axis
    sns.histplot(data=df, x='timepending', y='gasprice_gwei', bins=50)

    # Set the title and axis labels
    plt.title('Relationship between Total Gas Price in Private O.F. and Total Inclusion Time Per Block')
    plt.xlabel('Inclusion Time in Block (ms)')
    plt.ylabel('Total Gas Price by Private Order Flow for a Block (Gwei)')

    # Show the plot
    plt.show()

In [ ]:
def plot_gas_vs_inclusion(df):

    # Create a scatter plot with inclusion time on the x-axis and total gas spent in block on the y-axis
    plt.scatter(df['gasused'], df['timepending'])

    # Set the title and axis labels
    plt.title('Relationship between Total Gas Used in Private O.F. and Total Inclusion Time Per Block')
    plt.ylabel('Inclusion Time in Block (ms)')
    plt.xlabel('Total Gas Used by Private Order Flow for a Block (Gwei)')

    # Show the plot
    plt.show()

In [ ]:
# Fetch the confirmed transactions in a range:
sample_confirmed_df = new_pool[(new_pool['curblocknumber'] >= 17195490) & (new_pool['curblocknumber'] <= 17195500) & (new_pool['region'] == 'us-east-1') & (new_pool['status'] == 'confirmed')].copy()
big_confirmed_df = new_pool[(new_pool['region'] == 'us-east-1') & (new_pool['status'] == 'confirmed')].copy()

big_confirmed_df['detecttimemillis'] = pd.to_datetime(big_confirmed_df['detecttime']).astype(int) // 10**6
big_confirmed_df['gasprice_gwei'] = big_confirmed_df['gasprice'] * 1e-9

# define function to determine order flow
def determine_order_flow_documentation(row):
    if row['timepending'] == 0 and row['status'] == 'confirmed':
        return 'private'
    else:
        return 'public'

# define function to determine order flow
def determine_order_flow_burak(row):
    if row['timepending'] == 0 and row['status'] == 'confirmed':
        return 'private'
    elif pd.isna(row['timepending']) and row['status'] == 'confirmed':
        return 'private'
    elif row['timepending'] > 0 and row['status'] == 'confirmed':
        return 'public'
    else:
        return 'others'
    # else:
    #     return 'public'

# add the order flow 
big_confirmed_df['order_flow'] = big_confirmed_df.apply(determine_order_flow_burak, axis=1)

# take the private df
private_df = big_confirmed_df[big_confirmed_df['order_flow'] == 'private']
# take the public df
public_df = big_confirmed_df[big_confirmed_df['order_flow'] == 'public']

others_df = big_confirmed_df[big_confirmed_df['order_flow'] == 'others']

# group by curblocknumber and calculate total_gas_spent_in_block and total_gas_in_private_flow
grouped_private_df = private_df.groupby('curblocknumber').agg({'gasused': 'sum'}).reset_index()
# group by curblocknumber and calculate total_gas_spent_in_block and total_gas_in_private_flow
grouped_public_df = public_df.groupby('curblocknumber').agg({'timepending': 'median'}).reset_index()

# join grouped_private_df and grouped_public_df on 'curblocknumber'
merged_df = grouped_private_df.merge(grouped_public_df, on='curblocknumber')

## Grouped public.merge(grouped)

plot_gas_vs_inclusion(merged_df)
print(big_confirmed_df['curblocknumber'].nunique())

